# Лабораторная работа 2-3
Язык грамматики: итальянский

## Lab2

Написание контекстно-свободной грамматики для естественного или формального языка
Необходимо написать контекстно-свободную грамматику, описывающую фрагмент выбранного студентом языка. Объем грамматики: 15-20 правил. Грамматика может описывать как формальный, так и естественный язык.  

Критерии оценки:  
6 - полнота охвата выбранной группы (в рамках дюжины правил)  
4 - красота решения (субъективное мнение преподавателя)  


In [ ]:
#язык: итальянский

In [ ]:
S ::= VP S

VP ::= LEFTv V RIGHTv # местами поменять нельзя, так как подлежащее всегда перед глаголом, а причастия, герундии, зависимые от глагола VP  – после
LEFTv ::= eps | Subj* AdvP* PP* NP* AdvP* LEFTv # перед глаголом могут быть подлежащее, AdvP, PP, NP(все от 0 и больше в любом порядке)
RIGHTv ::= eps | VP* Ger* Part* NP* AdjP* PP* AdvP* RIGHTv

NP ::= LEFTn N RIGHTn # DetMin может идти только перед N, PartP только после N
LEFTn ::= DetMin PronMin AdjP LEFTn # все из этого может быть, может не быть
RIGHTn ::= PronMin AdjP PartP RIGHTn # N PartP: il libro letto (прочитанная книга)

Subj ::=  NP | PronMin
PP ::= Prep NP

AdjP ::= eps | AdjPmin AdjP # либо 0 AdjPmin, либо 1, либо много
AdjPmin ::= AdvMin* Adj | AdvMin* Adj PP | PP AdvMin* Adj  # (terribilmente) bello (a mio avviso) – ужасно красивый, на мой взгляд

PartP ::= eps | PartPMin PartP # причастия могут идти несколько подряд (il libro letto riletto – книга читанная перечитанная)
PartPMin ::= Part AdvP* PP* # letto molto velocemente sotto il piumone (прочитано очень быстро под одеялом)

AdvP ::= eps | Adv AdvP | AdvP PP... #  molto vicino a casa (очень близко к дому)

DetMin ::= eps | Det # артикль может быть подряд только 1
PronMin ::= eps | Pron PronMin # местоимения могут несколько подряд (Io te lo consiglio я тебе это советую)

Prep ::= preposition | prepositional_article | preposition Det
Adv ::= adverb
Pron ::= pronoun
V ::= verb
N ::= noun
Det ::= determiner
Adj ::= adjective
Part ::= participle
Ger ::= gerund

## Lab3

Необходимо написать программу с использованием контекстно-свободного парсера строк, которая бы реализовала разбор грамматики, написанной во второй лабораторной работе   

Критерии оценки:  

2 - рабочая программа  
2 - с использованием специальной библиотеки   
2 - парсер входного языка, который просто проверяет входное предложение  
2 - парсер входного языка, который совершает какие-то действия над входным предложением  
2 - красота решения (субъективное мнение преподавателя)  

### 1.Парсер входного языка, который просто проверяет входное предложение (NLTK + spacy)

#### Импорты

In [163]:
#!pip install spacy
#!python -m spacy download it_core_news_md


In [5]:
import nltk
import spacy
from nltk import CFG, RecursiveDescentParser
# модель для итальянского языка для pos теггирования входящего текста

#### POS теггирование предложения

In [146]:
def tag_sentence(sentence):
    doc = nlp(sentence)
    # получение POS-тегов для каждого слова
    pos_tags = [token.pos_ for token in doc if not token.is_punct]
    return pos_tags

sentence = "il gatto mangia il pesce"
tagged_sentence = tag_sentence(sentence)
tagged_sentence


['DET', 'NOUN', 'VERB', 'DET', 'NOUN']

#### Парсер

In [147]:
grammar = CFG.fromstring("""
    S -> Subj VP | S CONJ S
    Subj -> NP | PronP
    NP -> DetP N | DetP AdjP N | DetP N PP | DetP AdjP N PP | DetP N AdjP | NP CONJ NP | AdjP N AdjP | NP NP | NP PP |
    VP -> V RIGHTv | PronP V V |
    RIGHTv -> AdvP | NP | PP | NP PP | AdvP NP | AdjP | AdjP PP | AdjP PP NP | AdvP P VP | RIGHTv RIGHTv | V NP
    PronP -> PRON | PRON ADJ | PRON ADV | PRON ADJ ADV
    DetP -> Det AdjP | Det PROPN | Det | DetP DetP
    AdjP -> ADJ | ADJ AdjP | AdjP CONJ AdjP
    AdvP -> ADV | ADV AdvP | ADV PP
    PP -> P NP


    Det -> 'DET' |
    CONJ -> 'CCONJ' | "SCONJ" | ADV CONJ
    N -> 'NOUN' | 'PROPN'
    V -> 'VERB' | 'AUX' | PRON V
    ADV -> 'ADV'
    ADJ -> 'ADJ'
    P -> 'ADP'
    PRON -> 'PRON'


""")


In [148]:
def check_sentence_agreement(sentence, grammar):
    pos_tags = tag_sentence(sentence)
    parser = nltk.ChartParser(grammar)
    # попытка разбора предложения
    try:
        return any(True for _ in parser.parse(pos_tags))  # предложение соответствует грамматике
    except ValueError:
        return False #предложение не соответствует грамматике

In [149]:
# предложения разбираются:
"Il gatto mangia il pesce"
"Il gatto mangia il pesce e il cane beve l'acqua"
"Il grande gatto mangia il piccolo pesce"
"Il gatto mangia rapidamente il pesce"
"Il gatto mangia con il pesce"
"Il gatto grande mangia rapidamente il pesce e il cane piccolo beve con l'acqua"

"Il cane piccolo beve l'acqua e il gatto mangia il pesce."
"Il pesce mangia il gatto mentre il cane dorme."
"Il grande pesce mangia il piccolo gatto."
"Il pesce e il gatto giocano insieme."
"Il gatto dorme mentre il pesce nuota nel acquario."
"Il cane beve l'acqua mentre il gatto mangia il pesce."
"Il pesce piccolo e il grande gatto nuotano nel fiume."
"Il pesce mangia il gatto dopo che il cane mangia."
"Il pesce rosso e il gatto nero vivono insieme."
"Il gatto mangia il pesce fresco mentre il cane osserva."


"La bella ragazza legge un libro interessante nella biblioteca."
"Mia madre prepara deliziosi piatti italiani ogni giorno."
"I bambini giocano felici nel parco con il loro cane."
"Ogni mattina faccio una lunga passeggiata nel parco vicino casa."
"Ogni mattina mi sveglio presto per fare colazione."
"Il professore spiega la lezione con molta chiarezza agli studenti."
"La dolce melodia della chitarra riempie l'aria della serata."
"So un regalo speciale per il compleanno di mia sorella."


"La casa è grande e luminosa."
"La piazza del mercato è piena di bancarelle e persone."
"La torta al cioccolato è la mia tarta preferita."
"Il mio gatto nero dorme sul divano."
"Le montagne innevate sono uno spettacolo meraviglioso."
"I bambini felici giocano nel parco sotto il sole."
"Il treno per Roma parte dalla stazione principale."
"Il grande gatto nero mangia il delizioso pesce fresco velocemente nella cucina luminosa e spaziosa di mia nonna."
"Lo devo fare"

'Lo devo fare'

In [150]:
# проверка предложения на согласованность с грамматикой
sentence = "Io voglio suonare il pianoforte"
is_agree = check_sentence_agreement(sentence, grammar)
print("True" if is_agree else "False")

# токенизация и POS-теггирование предложения
pos_tags = tag_sentence(sentence)
print("\nPOS-tagged:", pos_tags)

True

POS-tagged: ['PRON', 'AUX', 'VERB', 'DET', 'NOUN']


### 2. Парсер входного языка, который превращает утвердительное предложение в отрицательное

In [161]:
# non ставится перед глаголом
# если перед глаголом еще стоит местоимение (указательное/возвратное), то non перед ним
def negate_sentence_if_agree(sentence):
    # Проверка соответствия предложения грамматике
    is_agree = check_sentence_agreement(sentence, grammar)
    if is_agree:
        print("Предложение:", sentence)
        print("Предложение соответствует грамматике")
    else:
        print("Предложение не соответствует грамматике:", sentence)
        return

    doc = nlp(sentence)
    negated_sentence = []
    negation_added = False

    for token in doc:
        # если токен - не личное местоимение и следующий токен - глагол
        if token.pos_ == "PRON" and token.text.lower() not in ["io", "tu", "lui", "lei", "noi", "voi", "loro"] and token.i + 1 < len(doc) and (doc[token.i + 1].pos_ == "VERB" or doc[token.i + 1].pos_ == "AUX") and not negation_added:
            negated_sentence.append("non")
            negation_added = True
            negated_sentence.append(token.text)
        # если токен - глагол и отрицание ещё не добавлено
        elif (token.pos_ == "VERB" or token.pos_ == "AUX") and not negation_added:
            negated_sentence.append("non")
            negated_sentence.append(token.text)
            negation_added = True
        elif negation_added:
            negated_sentence.append(token.text)
        else:
            negated_sentence.append(token.text)

    negated_result = " ".join(negated_sentence)
    print("Отрицание:", negated_result)



In [162]:
sentences = [
    "Noi ti crediamo",
    "La casa è grande",
    "Il grande pesce mangia il piccolo gatto",
    "Io voglio suonare il pianoforte",
    "Io lo voglio fare"
]

for sentence in sentences:
    negate_sentence_if_agree(sentence)
    print("\n")


Предложение: Noi ti crediamo
Предложение соответствует грамматике
Отрицание: Noi non ti crediamo


Предложение: La casa è grande
Предложение соответствует грамматике
Отрицание: La casa non è grande


Предложение: Il grande pesce mangia il piccolo gatto
Предложение соответствует грамматике
Отрицание: Il grande pesce non mangia il piccolo gatto


Предложение: Io voglio suonare il pianoforte
Предложение соответствует грамматике
Отрицание: Io non voglio suonare il pianoforte


Предложение: Io lo voglio fare
Предложение соответствует грамматике
Отрицание: Io non lo voglio fare


